# Notes & Code

## Chapter: Scaling with generators

Let's start with iterations, iteration is simply pass over each element in a sequence doing some operation on that. It's largelly used in for loops. But when a for loop is used, what happens under the hood? <br>
So actually python uses a built-in function called `iter()` transforming a sequence into a `iterator`, what points to next element each iteration. <br> 

- iterator: An iterator is like a moving pointer over the collections.

In [2]:
numbers = [1, 2, 3, 4, 5]

for num in numbers: print(num)

1
2
3
4
5


In [5]:
numbers = [1, 2, 3, 4, 5]

numbers_iter = iter(numbers)

for num in numbers_iter: print(num)
    
print(f'numbers type: {type(numbers)}')
print(f'numbers_iter type {type(numbers_iter)}')

1
2
3
4
5
numbers type: <class 'list'>
numbers_iter type <class 'list_iterator'>


So I have this situation for example in a for loop what python actually does is get some sequence and transform it into a iterator. And how python do that?<br>
Simple!<br>

There is a built-in function called iter() that get a sequence and return a iterator like showed previously. And how does iter() get the iterator?<br>

Many ways to do that but one specific is define the magic dunder method `__iter__`. In this point we have to define the difference between iterator and iterable.<br>

- iterable: Are objects that return an iterator when passed through iter() function.
- iterator: Are the returns of iter() function, an object that have give one element by time.

In [1]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [5]:
next(names_it)

StopIteration: 

Analisando mais detalhadamente a operação iter() sobre um objeto iterável, podemos utilizar outra built-in function chamada next() para acessar o próximo elemento do iterador. aplicando next() sucessivas vezes, consigo acessar todos os elementos da sequencia, até que ao não ter mais elementos, ocorre um raise exception de um erro do tipo `StopIteration`.

In [57]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [21]:
# neste caso, tenho também a presença de um valor default onde ao invés de um raise exception, terei obterei o valor default.
# este valor default continua ocorrendo indefinidamente ao acessar o iterator desta forma.

next(names_it, "Rodrigo")

'Rodrigo'

Vamos agora, começar a analisar onde esta definição de iteradores e iteráveis podem trazer ganho em sua utilização.

In [25]:
# Definição de uma função que calcula uma lista de quadrados de numeros, um a um recebendo como parametro o limite superior 
# da lista

def calculate_squares(__max):
    
    squares = list()
    
    for num in range(__max):
        
        squares.append(num ** 2)
        
    return squares

In [28]:
calculate_squares(4)

[0, 1, 4, 9]

Reparamos que neste caso passando para função o valor máximo de elementos que quero calcular o quadrado, é retornado uma lista de valores. Isto significa que preciso ocupar espaço na memória com uma lista de quadrados calculados. Ou seja, dependendo do numero de elementos este valor pode rapidamente explodir a memória. Imagine se quiser uma lista com 1 milhão de quadrados ou mais.<br>
Como comentado anteriormente, se uma classe definir o método mágico `__iter__` e o método `__next__` é possível obter um iterador passando este objeto pela função iter() e neste caso, também como visto anteriormente, não seria criada uma lista de valores e sim ocorreria a obtenção de um valor da sequencia por vez conforme necessário.<br>
Vamos ver a seguir como fazer esta definição utilizando orientação a objetos.

In [295]:
class SquareIterator():
    
    def __init__(self, max_value):
        
        self.max_value = max_value
        self.current_value = 0
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        
        if self.current_value >= self.max_value:
            
            raise StopIteration
            
        current_squared_value = self.current_value ** 2
        
        self.current_value += 1
        
        return current_squared_value

In [30]:
for num in SquareIterator(3):
    
    print(num)

0
1
4


In [31]:
for num in calculate_squares(3):
    
    print(num)

0
1
4


Utilizando tanto a classe como a função, quanto a classe eu consigo resolver o problema de iterar sobre uma lista de quadrados de números entre 0 e um valor máximo.
A diferença é que no caso da classe, eu não precisar gerar uma lista completa a cada chamada next, o objeto altera seu estado através do retorno de uma nova instância com seus atributos atualizados até atingir o limite.

Interessante, entramos agora em um novo questionamento...<br>
Ok, interessante essa criação da classe onde definindo os métodos `__iter__` e `__next__` consigo criar meu iterator sem precisar criar de fato uma sequencia inteira, acessando os elementos um a um conforme for preciso.<br>
A dúvida é, existe forma mais fácil e prática de fazer isso?<br>
E a resposta é sim!

### Generator Functions

As generate functions pelo que estou lendo tem muitas funcionalidades, mas a principal explorada neste texto é exatamente a praticidade na geração de iterators.<br>
Estas funções se parecem muito com uma função tradicional mas ao invés de ter um `return` ocorre uma nova palavra-chave, `yield`. E é exatamente este cara que faz toda a diferença.

In [1]:
def gen_nums():
    
    n = 0
    
    while n < 4:
        
        yield n
        
        n += 1

In [4]:
for num in gen_nums():
    
    print(num)

0
1
2
3


In [5]:
sequence = gen_nums()
type(sequence)

generator

Vamos as definições:

- `GENERATOR FUNCTION:` - É uma função na qual eu defino retorno através da palavra-chave yield e não da palavra `return`.

- `GENERATOR OBJECT:` - É o objeto retornado pela GENERATOR FUNCTION, e na real, ele é um iterator onde posso aplicar a built-in function next() e utilizar em for loops por exemplo.

In [10]:
next(sequence)

StopIteration: 

Aqui temos uma definição interessante sobre as GENERATOR FUNCTIONS e sua comparação com as regular functions.<br>
Enquanto funções regulares podem apresentar vários pontos de saída (return statement), elas apresentam apenas um ponto de entrada. Ou seja, toda vez que uma função regular é chamada, ela roda desde sua primeira linha.<br>
No caso das GENERATOR FUNCTIONS, cada yield statement funciona como ponto de saída e entrada. Ou seja, ao chamar a função uma primeira vez, ela roda até o primeiro yield, ao rodar a função novamente ela começa a rodar exatamente na linha seguinte ao yield retornado anteriormente. 

__Obs:__ Não é necessário dar um raise em uma excessão (StopIteration), no caso do GENERATOR OBJECT retornado por uma GENERATOR FUNCTION, ele jé é implementado nativamente.<br>

Vamos fazer alguns testes:

In [11]:
def gen_function():
    
    yield "Rodrigo Bernardo Medeiros"

In [33]:
sequence = gen_function()

In [35]:
next(sequence)

StopIteration: 

Seria uma prática interessande utilizar generator functions para definir fluxos de desenvolvimento, ou para operar parcialmente sobre valores?

In [36]:
def split_names(complete_name):
    
    names = complete_name.split()
    
    for name in names:
        
        yield name

In [42]:
names = [
    'rodrigo bernardo medeiros', 
    'adriane bernardo medeiros', 
    'roberval mauricio cesar'
]

first_name = list()
second_name = list()
third_name = list()

for name in names:
    
    separated_names = split_names(name)
    
    first_name.append(next(separated_names))
    second_name.append(next(separated_names))
    third_name.append(next(separated_names))

In [44]:
first_name

['rodrigo', 'adriane', 'roberval']

In [45]:
second_name

['bernardo', 'bernardo', 'mauricio']

In [46]:
third_name

['medeiros', 'medeiros', 'cesar']

Neste exemplo acima, eu utilizo uma GENERATOR FUNCTION para tratar determinado informação.

Vamos retornar ao exemplo do quadrado dos números, reformulando a solução utilizando uma GENERATOR FUNCTION.

In [47]:
def gen_square_numbers(max_value):
    
    current_value = 0
    
    while current_value < max_value:
        
        yield current_value ** 2
        
        current_value += 1

In [48]:
for num in gen_square_numbers(4):
    
    print(num)

0
1
4
9


Muito bom, neste caso foi possível reescrever a classe escrita anteriormente que definia diretamente um itarator por uma GENERATOR FUNCTION que retorna um GENERATOR OBJECT que no fundo também é um ITERATOR.
Repara-se pela verbosidade de cada código que utilizar GENERATOR FUNCTIONS parece ser muito mais natural, fácil de ler, manter e escalar.

### Generator Patterns and scalable Composability

Agora vamos explorar um pouco mais as GENERATOR FUNCTIONS.

In [123]:
def matching_lines_from_file(path, pattern):
    
    with open(path) as handle:
        
        for line in handle:
            
            if pattern in line:
                
                yield line.rstrip('\n')

Neste exemplo, escrevi uma função para ler um arquivo e retornar um GENERATOR OBJECT que itera sobre as linhas que contém determinado padrão de caracteres.

In [136]:
path = r'./src/log_file.txt'
pattern = 'WARNING'

for line in matching_lines_from_file(path, pattern):
    
    print(line)

Através da GENERATOR FUNCTION, consegui iterar somente pelas linhas de interesse sem precisar trazer para meu código a complexidade de encontrar o padrão. Resolvo isso através do iterator. Posso escrever essas funções como se fossem uma espécie de filtro sem precisar construir uma sequencia para alocar a informação.

### Algumas boas práticas neste desenvolvimento:

- Utilizar `with` para abrir o arquivo, é uma boa prática que garante que o arquivo será fechado assim que o código passar pelo bloco de contexto.

- Neste exemplo, utilizei o modo pythonico de iterar por linhas em um arquivo. Utilizando o for line in file, aproveito este ponto para lembrar de registrar e não mais utilizar o readlines(), este método carrega o arquivo inteiro e com essa joga fora toda minha estratégia de utilizar menos memória em meus desenvolvimentos. Ou seja, jogo fora minha escalabilidade.

Vamos avançar em nossa função para analisar nosso arquivo de log, agora além de pegar somente as linhas com o padrão WARNING, quero também parsear a informação da linha em um dicionário com duas chaves, `level` para escrever o tipo de mensagem de log e `message` para escrever a mensagem efetivamente:


In [137]:
def parse_log_records(lines):
    
    for line in lines:
        
        level, message = line.split(':', 1)
        
        yield {
            "level": level,
            "message": message
        }

In [138]:
valid_lines = matching_lines_from_file(path, pattern)

for parsed_info in parse_log_records(valid_lines):
    
    print(parsed_info)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


Neste caso combinei duas funções para tratar problemas distintos, ambas são GENERATOR FUNCTIONS que retornam GENERATOR OBJECTS. pelo livro que estou estudando esse tratamento define a composição de escalabilidade. Ou seja, é sempre importante pensar na implementação a nível desses pequenos detalhes, como escalar, como evitar grandes consumos de memória.
Mais um detalhe de implementação, a função matching_lines_from_file, recebe um caminho e padrão para executar duas tarefas, uma ler os arquivos e outra retornar um GENERATOR OBJECT contendo somente as linhas de interesse. Neste ponto vamos fazer uma refatoração para que as interfaces estejam alinhadas e ambas as funções possam receber iteradores e operar sobre eles.

In [143]:
# função responsável por ler um arquivo retornando um GENERATOR OBJECT que itera sobre todas as linhas.

def lines_from_file(path):
    
    with open(path) as handle:
        
        for line in handle:
            
            yield line.rstrip('\n')
            
def matching_lines(lines, pattern):
    
    for line in lines:
        
        if pattern in line:
            
            yield line

In [140]:
lines_from_file = lines_from_file(path)
correct_lines = matching_lines(lines_from_file, 'WARNING')

for line in parse_log_records(correct_lines):
    
    print(line)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


### Teste utilizando `yield from`

Esta composição de palavras-chave é utilizada quando quero encadear duas generate functions. Vou desenvolver um exemplo baseado na leitura do arquivo com log.

In [150]:
def matching_lines_from_files(path, pattern='WARNING'):
    
    all_lines = lines_from_file(path)
    
    yield from matching_lines(all_lines, pattern)

In [152]:
for line in parse_log_records(matching_lines_from_files(path, pattern='WARNING')):
    
    print(line)

{'level': 'WARNING', 'message': ' Disk usage exceeding 85%'}
{'level': 'WARNING', 'message': ' Almost out of beer'}


### Python está cheio de iteradores

Vamos ver alguns exemplos:

In [167]:
calories = {
    "apple": 95,
    "slice of bacon": 43,
    "cheddar cheese": 113,
    "ice cream": 15
}

items = calories.items()

In [170]:
len(items)

4

In [171]:
iter(items)

In [173]:
('apple', 95) in items

True

In [177]:
for key, value in items: print(f'{key}: {value}')
for ind, item in enumerate(items): print(f'{ind + 1} - {item}')

apple: 95
slice of bacon: 43
cheddar cheese: 113
ice cream: 15
1 - ('apple', 95)
2 - ('slice of bacon', 43)
3 - ('cheddar cheese', 113)
4 - ('ice cream', 15)


Além desse exemplo de iterador, no livro são citados diversos outros exemplos de iterators em python, como os resultados da utilização de map, filter e zip. Vou codar um pouco com esses caras pra ver no que dá.

In [234]:
str_numbers = ['1', '2', '3', '4']

def change_type(number):
    
    return int(number)

int_numbers = map(change_type, str_numbers)

type(int_numbers)

map

A aplicação de uma função map, retorna um objeto do tipo map que adivinhem só, é um iterator.

In [235]:
def filter_test(number):
    
    if number > 2:
        
        return number
    

filter_numbers = filter(filter_test, int_numbers)
type(filter_numbers)

filter

Fazendo um paralelo com a aplicação do map, ao aplicar o filter obtenho um objeto do tipo filter que também é um iterator. É importante saber que isso existe e sempre pensar nesse gerenciamento de memória ao desenvolver as soluções com isso vamos ficando cada vez mais afiados. Lembrando que enquanto a gente só conhece martelo, todo problema é prego. Conhecer novas formas de resolver problemas nos torna mais criativos

### Iterator protocol

objetos que são considerados iterators se seguem o seguinte protocolo:

1) Tem a implementação de um método `__next__` que não tem parametros.<br>
2) O método `__next__` é responsável por atualizar o estado do objeto para o próximo item na sequencia.<br>
3) Após todos os elementos serem produzidos, o método `__next__` é responsável por dar um raise exception no erro StopIteration.<br>
4) Ocorre a definição de um método `__iter__` que não recebe parametros e retorna o próprio elemento. Liretamente um método com um return self dentro.<br>

### Definição de um objeto iterável.

In [240]:
import pandas as pd

In [264]:
content = {
    'names': [
        'rodrigo', 
        'renata',
        'isabella', 
        'gabriella'
    ],
    'age': [
        35,
        34,
        2,
        0
    ]
}

family = pd.DataFrame(content)

In [265]:
family.to_dict(orient='records')

[{'names': 'rodrigo', 'age': 35},
 {'names': 'renata', 'age': 34},
 {'names': 'isabella', 'age': 2},
 {'names': 'gabriella', 'age': 0}]

In [285]:
for ind, item in family.iterrows():
    
    print(dict(item))

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [287]:
family.loc[0]

names    rodrigo
age           35
Name: 0, dtype: object

In [288]:
class DataFrame():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe.reset_index(drop=True)
        self._index = 0
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        
        try:
            
            return dict(self._dataframe.loc[self._index])
        
        except:
            
            raise StopIteration
        
        finally:
            
            self._index += 1

In [289]:
# implementing __iter__ metho to define an iterable object, this method must return a iterator in this case I just apply 
# the built-in function iter() in the list.

class IterRowsDataFrame():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe
        
    def __iter__(self):
        
        return iter(self._dataframe.to_dict(orient='records'))

In [290]:
# Just to solve the problem in othet way, in this case instead of use iter(), I defined the __iter__ method as a generator
# function which returns a generator object.

class IterRowsDataFrameGeneratorFunction():
    
    def __init__(self, dataframe):
        
        self._dataframe = dataframe
        
    def __iter__(self):
        
        index = 0
        
        while index < len(self._dataframe):
        
            yield self._dataframe.to_dict(orient='records')[index]
            
            index += 1

In [291]:
dict_rows_family = DataFrame(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [292]:
dict_rows_family = IterRowsDataFrame(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [293]:
dict_rows_family = IterRowsDataFrameGeneratorFunction(family)
for item in dict_rows_family: print(item)

{'names': 'rodrigo', 'age': 35}
{'names': 'renata', 'age': 34}
{'names': 'isabella', 'age': 2}
{'names': 'gabriella', 'age': 0}


In [277]:
teste = DataFrame(family)
next(teste)

{'names': 'rodrigo', 'age': 35}

In [278]:
teste = IterRowsDataFrame(family)
next(teste)

TypeError: 'IterRowsDataFrame' object is not an iterator

In [279]:
teste = IterRowsDataFrameGeneratorFunction(family)
next(teste)

TypeError: 'IterRowsDataFrameGeneratorFunction' object is not an iterator

Como podemos ver nestes exemplos, a classe DataFrame é um iterator pela definição dos métodos necessários (`__iter__`, `__next__`), enquanto que as demais são apenas iterables definindo somente o método `__iter__`.

# CREATING COLLECTIONS WITH COMPREHENSIONS

# CRIANDO COLEÇÕES COM COMPREENSÕES

List comprehension é uma forma declarativa de alto nível para criar listas em python.

In [1]:
squares = [n * n for n in range(6)]
print(squares)

[0, 1, 4, 9, 16, 25]


Isto é exatamente equivalente ao seguinte:

In [2]:
squares = list() # ou squares = []

for n in range(6):
    
    squares.append(n * n)
    
print(squares)

[0, 1, 4, 9, 16, 25]


Observe que no primeiro exemplo, o que você digita está declarando que tipo de lista você quer, enquanto que no segundo está especificando como cria-la. 
Por isso que nós dizemos que é uma forma alto nível e declarativa:Funciona como se vocês estivesse definindo que tipo de lista quer criar, e depois, deixa o Python descobrir como construi-la. 

Python permite escrever outros tipos de `comprehensions` além das listas. Segue um exemplo de `dictionary comprehension`:

In [3]:
blocks = { n: "x" * n for n in range(5)}
print(blocks)

{0: '', 1: 'x', 2: 'xx', 3: 'xxx', 4: 'xxxx'}


Isto é exatamente equivalente a fazer o seguinte:

In [4]:
blocks = dict()

for n in range(5):
    
    blocks[n] = 'x' * n
    
print(blocks)

{0: '', 1: 'x', 2: 'xx', 3: 'xxx', 4: 'xxxx'}


Os principais benefícios das `comprehensions` são legibilidade e manutenibilidade. Muitas pessoas, acham elas muito legíveis; Mesmo desenvolvedores que nunca encontraram elas antes, podem na maioria das vezes adivinhar corretamente o que significa.
E além disso, há um benefício cognitivo mais profundo: Uma vez que você praticar um pouco, vai achar que pode escreve-las com pouquíssimo esforço mental - mantendo sua atenção livre para outras tarefas.

## List Comprehensions

__`List comprehension` é a mais usada e util tipo de `comprehension`, e essencialmente é uma forma de criar e popular uma lista.__ Sua estrutura segue a seguinte expressão:

[ __EXPRESSION__ for __VARIABLE__ in __SEQUENCE__ ]

*EXPRESSION* é qualquer expressão python, embora em compreensões, esta expressão tipicamente contenha uma variável. Esta variável é declarada no campo *VARIABLE*. *SEQUENCE* define a fonte dos valores pelos quais a variável itera, criando uma sequencia final de valores calculados.

In [17]:
squares = [n * n for n in range(6)]
print(type(squares))
print(squares)

<class 'list'>
[0, 1, 4, 9, 16, 25]


Observe que o resultado é uma lista regular. Em __squares__, a expressão é n\*n; a variável é n; e a sequencia fonte é range(6). A sequencia é um objeto do tipo range; Na verdade, neste ponto podemos utilizar qualquer iterável, outra lista ou tuple, um generator object, ou qualquer outra coisa.
<br>
A parte da expressão pode ser qualquer que se reduz a um valor:
<br>
- Expressões aritiméticas
- Uma chamada de função do tipo f(m), utilizando m como variável.
- Uma operação de corte (como s[::-1], para reverter uma string).
- Chamadas de métodos (foo.bar(), iterando sobre uma sequencia de objetos).
- E muito mais...

__Alguns exemplos:__

In [21]:
# Primeiro definimos algumas sequencias fonte:

pets = 'dog parakeet cat llama'.split()
numbers = [9, -1, -4, 20, 11, -3]

# Uma função para chamar
def repeat(s):
    
    return s + '_' + s

# Agora, alguns exemplos de list comprehensions:

print([2 * m + 3 for m in range(10, 20, 2)])
print(sorted([abs(num) for num in numbers]))
print([10 - x for x in numbers])
print([pet.lower() for pet in pets])
print(["The " + pet for pet in pets])
print([repeat(pet) for pet in pets])

[23, 27, 31, 35, 39]
[1, 3, 4, 9, 11, 20]
[1, 11, 14, -10, -1, 13]
['dog', 'parakeet', 'cat', 'llama']
['The dog', 'The parakeet', 'The cat', 'The llama']
['dog_dog', 'parakeet_parakeet', 'cat_cat', 'llama_llama']


Observe como todas se encaixam na mesma estrtura. Todos tem as palavras-chave __for__ e __in__; Que são requeridas  pelo Python, em qualquer tipo de compreensão que você possa escrever. Elas são intercaladas entre três campos: a expressão; a variável; e a sequencia fonte.<br>

A ordem dos elementos na lista final é determinado pela ordem da sequencia fonte. Porém é possível filtrar adicionando uma clausula *if*.

In [24]:
def is_palindrome(s):
    
    return s == s[::-1]

pets = 'dog parakeet cat llama'.split()
numbers = [9, -1, -4, 20, 11, -3]
words = 'bib bias dad eye deede tooth'.split()

print([n*2 for n in numbers if n % 2 == 0])
print([pet.upper() for pet in pets if len(pet) == 3])
print([n for n in numbers if n >0])
print([word for word in words if is_palindrome(word)])

[-8, 40]
['DOG', 'CAT']
[9, 20, 11]
['bib', 'dad', 'eye']


A estrutura é:

[EXPR for VAR in SEQUENCE if CONDITION]

Onde *CONDITION* é a expressão que avalia para Verdadeiro ou Falso, dependendo da variável. Note que também podemos utilizar uma função aplicada a variável (is_palindrome(word)), ou algo mais complexo (len(pet) == 3). Optando por usar uma função pode melhorar a legibilidade, e também permite aplicar uma lógica de filtro que não se encaixa em somente uma linha.

Uma `list comprehension` sempre deve ter a palavra "for", mesmo que a expressão seja a própria variável. Por exemplo, quando dizemos:

In [25]:
[word for word in words if is_palindrome(word)]

['bib', 'dad', 'eye']

As vezes as pessoas pensam que `word for word in words` parece redundante (e é), e tentam encurtar a sentença. Porém, isso não funciona, como apresentado no exemplo a seguir:

In [26]:
[word in word if is_palindrome(word)]

SyntaxError: invalid syntax (<ipython-input-26-42bedfba4293>, line 1)

### Formatando Para Legibilidade (e mais)

`List Comprehensions` realisticas, tendem ser muito longas para encaixar bem em apenas uma linha. E elas são compostas de partes lógicas distintas, que podem variar independentemente conforme o código evolui. Isto cria alguns inconvenientes, que são resolvidos por um fato muito conveniente: As regras normais de espaços em branco são suspensas dentro de colchetes. Você pode explorar isso e fazer com as `list comprehensions` sejam mais legíveis e manuteníveis, divindo-as através de múltiplas linhas.

In [27]:
def double_short_words(words):
    
    return [
        word + '_' + word
        for word in words
        if len(word) < 5
    ]

In [28]:
print(double_short_words(words))

['bib_bib', 'bias_bias', 'dad_dad', 'eye_eye']


O que foi feito aqui foi dividir a `comprehension` através de linhas separadas. Você pode, e deve, fazer isso com qualquer `comprehension`. Isso é ótimo por várias razões, a mais importante sendo o ganho instantâneo na legibilidade. Esta `comprehension` tem três ideias separadas expressas dentro dos colchetes: a expressão (word + word); a sequencia (for word in words); e a clausula de filtro (if len(word) < 5). Estes são aspectos lógicos separados, e fazendo a divisão através das linhas, toma-se menos esforço cognitivo para um humano ler e entender quando comparado a versão escrita em somente uma linha. Com isso a lista está previamente parseada para você, conforme você lê o código.

Existe um outro benefício: Controle de versão e revisão de código são mais fáceis de se fazer. Imagine que você e eu somos desenvolvedores no mesmo time, trabalhando na mesma base de código em diferentes branchs. No meu branch, eu altero a expressão `word + '_' + word` para `word + '_' + word + '_' + word`; no seu, você muda o limiar para "len(word) < 7". Se a `comprehension` estiver na mesma linha, as ferramentas de versionamento de código vão identificar isso como um conflito de fusão, e estes conflitos terão que ser resolvidos manualmente. Mas uma vez que a `list comprehension` esteja dividida entre várias linhas, nossa ferramenta de controle de versão vai automaticamente ser capaz de juntar todas as modificações sem conflito. E se nós estivermos fazendo uma revisão de código, como deveriamos, o revisor pode identificar com precisão a mudança imediatamente, sem ter que percorrer linhas por linha pensando.

### Múltiplas Fontes e Filtros

Você pode ter várias clausulas `for VAR in SEQUENCE`. Isto permite que você construa listas com pares, trios, etc... a partir de uma ou mais sequências fonte.

In [31]:
colors = 'orange purple pink'.split()
toys = 'bike basketball skateboard doll'.split()

[
    color + '_' + toy
    for color in colors
    for toy in toys
]

['orange_bike',
 'orange_basketball',
 'orange_skateboard',
 'orange_doll',
 'purple_bike',
 'purple_basketball',
 'purple_skateboard',
 'purple_doll',
 'pink_bike',
 'pink_basketball',
 'pink_skateboard',
 'pink_doll']

Todo par vindo dessas duas fontes, *colors* e *toys*, é utilizado para calcular um valor na lista final. Esta lista final tem 12 elementos, o produto da dimensão das duas listas fonte.

Eu quero que você note que as duas clausulas `for` são independentes entre si; *colors* e *toys* são duas listas não relacionadas. Utilizar múltiplas clausulas `for` pode as vezes tomar uma forma diferente, onde elas são mais independentes.

Considere este exemplo:

In [40]:
ranges = [range(1, 7), range(7, 12), range(12, 21)]

[
    float(num)
    for subrange in ranges
    for num in subrange
]

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0]

A sequencia fonte - "ranges" - é uma lista de objetos do tipo range. Agora, esta `list comprehension` tem duas clausulas novamente. Mas observe que uma depende da outra. A fonte da segunda é uma variável da primeira!

Este exemplo não é como o exemplo de brinquedos coloridos, onde as clausulas são independentes umas das outras. Quando encadeados desta forma a ordem é importante.

In [41]:
[
    float(num)
    for num in subrange
    for subrange in ranges
]

NameError: name 'subrange' is not defined

Python avalia a `list comprehension` da esquerda para a direita. Se a primeira clausula é `for num in subrange`, neste momento, subrange não está definido ainda. Então, você deve colocar `for surange in ranges` primeiro. Você pode encadear mais que duas clausula `for` juntas; a primeira vai somente referenciar uma fonte definida anteriormente, e as outras podem usar as fontes definidas nas clausulas `for` anteriores, como a subrange mostrada no exemplo.

Beleza, isto é para clausulas `for` encadeadas. Se as clausulas são independentes, a ordem importa? A resposta é sim, importam porém de uma forma diferente. Qual a diferença entre essas duas duas `list comprehensions`.

In [42]:
colors = 'orange purple pink'.split()
toys = 'bike basketball skateboard doll'.split()

In [49]:
[
    color + ' ' + toy
    for color in colors
    for toy in toys
]

['orange bike',
 'orange basketball',
 'orange skateboard',
 'orange doll',
 'purple bike',
 'purple basketball',
 'purple skateboard',
 'purple doll',
 'pink bike',
 'pink basketball',
 'pink skateboard',
 'pink doll']

In [50]:
[
    color + ' ' + toy
    for toy in toys
    for color in colors
]

['orange bike',
 'purple bike',
 'pink bike',
 'orange basketball',
 'purple basketball',
 'pink basketball',
 'orange skateboard',
 'purple skateboard',
 'pink skateboard',
 'orange doll',
 'purple doll',
 'pink doll']

A ordem aqui não importa na mesma forma que as clausulas `for` encadeadas, onde você tem que colocar as coisas em certa ondem, ou seu programa não vai rodar. Aqui, você tem uma escolha. E esta escolha a ordem dos elementos na lista final. O primeiro elemento em cada um dos arranjos é "orange bike". E observe que o segundo elemento é diferente. Pense um momento, e pergunte a você mesmo: Por que? Por que o primeiro elemento é o mesmo em ambas as listas? E porque somente a partir do segundo elemento o arranjo é diferente?

Isto tem haver com qual sequencia é mantida constante enquanto a outra varia. É a mesma lógica que aplicamos em loops `for` aninhados.

In [52]:
# Primeira configuração:

build_colors_toys = []

for color in colors:
    
    for toy in toys:
        
        build_colors_toys.append(color + " " + toy)
        
        
print(build_colors_toys[0])
print(build_colors_toys[1])

orange bike
orange basketball


In [53]:
# Segunda configuração:

build_toys_colors = []

for toy in toys:
    
    for color in colors:
        
        build_toys_colors.append(color + " " + toy)
        
print(build_toys_colors[0])
print(build_toys_colors[1])

orange bike
purple bike


A segunda clausula `for` na `list comprehension` corresponde ao `for loop` mais interno. Os valores variam através de seu range mais rapidamente do que o loop externo.

E além de poder utilizar múltiplas clausulas `for`, você pode ter mais de uma clausula `if`, para multiplos níveis de filtro. Basta escrever várias clausulas em sequencia.

In [73]:
numbers = [9, -1, -4, 20, 17, -3]

odd_positives = [
    num for num in numbers
    if num % 2 == 1
    if (num >0)
]

print(odd_positives)

[9, 17]


Aqui, eu coloquei cada clausula `if` em sua própria linha, pela legibilidade - mas eu poderia colocar ambas em uma linha apenas. Quando você tiver mais de uma clausula `if`,  cada elemento deve atender todos os critérios para fazer parte da lista final. Em outras palavras, clausulas `if` são do tipo `and` e não `or`.

E se você quiser fazer uma clausula `or` - para incluir elementos que atendem pelo menos uma das clausulas `if`, omitindo apenas aquelas que não atendem ambas? `list comprehension` não permite fazer isso diretamente. A mini-linguagem para escrever `comprehensions` não é tão expressiva quando o próprio Python, e existem listas que você precisa construir que não podem ser expressas como uma `comprehension`.

Mas as vezes, você pode contornar um pouco através da definição de funções. Por exemplo, como você poderia construir um filtro baseado na clausula de um número ser múltiplo de 2 __ou__ 3.

In [74]:
numbers = [9, -1, -4, 20, 11, -3]

def is_mul_of_2_or_3(num):
    
    return (num % 2 == 0) or (num % 3 == 0)

[
    num for num in numbers
    if is_mul_of_2_or_3(num)
]

[9, -4, 20, -3]

__COMENTÁRIO PESSOAL__

Uma outra alternativa é escrever uma clausula `if` mais complexa, composta por múltiplos testes.

In [83]:
[
    num for num in numbers
    if (num % 2 == 0) | (num % 3 == 0)
]

[9, -4, 20, -3]

Essa limitações serão discutidas melhor posteriormente.

Podemos utilizar multiplas clausulas `for` e `if` juntas:

In [85]:
weights = [0.2, 0.5, 0.9]
values = [27.5, 13.4]
offsets = [4.3, 7.1, 9.5]

from collections import namedtuple

infos = namedtuple('WeightInfo', 'weigth value offset')

[
    infos(weight, value, offset)
    for weight in weights
    for value in values
    for offset in offsets
    if offset >5.0
    if weight * value < offset
]

[WeightInfo(weigth=0.2, value=27.5, offset=7.1),
 WeightInfo(weigth=0.2, value=27.5, offset=9.5),
 WeightInfo(weigth=0.2, value=13.4, offset=7.1),
 WeightInfo(weigth=0.2, value=13.4, offset=9.5),
 WeightInfo(weigth=0.5, value=13.4, offset=7.1),
 WeightInfo(weigth=0.5, value=13.4, offset=9.5)]

__COMENTÁRIO PESSOAL__

O exemplo do livro não contém namedtuple, e sim uma lista de tuples como apresentado a seguir.

In [82]:
[
    (weight, value, offset)
    for weight in weights
    for value in values
    for offset in offsets
    if offset > 5.0
    if weight * value < offset
]

[(0.2, 27.5, 7.1),
 (0.2, 27.5, 9.5),
 (0.2, 13.4, 7.1),
 (0.2, 13.4, 9.5),
 (0.5, 13.4, 7.1),
 (0.5, 13.4, 9.5)]

A única regra é que a primeira clausula `for` deve vir antes da primeira clausula `if`. Tirando isso, você pode intercalar as clausulas `for` e `if` em qualquer ordem, embora a maioria das pessoas parecem achar mais legível agrupar todas as clausulas `for` primeiro, e depois agrupar todas as clausulas `if` no final.

### `Comprehensions` e `Generators`

`List comprehensions` cria listas:

In [86]:
squares = [n*n for n in range(6)]
print(type(squares))

<class 'list'>


Quando você precisa de uma lista, isto é ótimo, mas as vezes você não precisa de uma lista, você poderia preferir algo mais escalável. Esta parece uma situação próxima a comentada no capítulo sobre `generators`.

In [89]:
NUM_SQUARES = 10*1000*1000
many_squares = [n*n for n in range(NUM_SQUARES)]

def do_something(num):
    
    return None

for number in many_squares:
    
    do_something(number)    

Toda a lista many_squares precisa ser criada - toda memória alocada, e cada elemento calculado - antes que a função `do_something` possa ser chamada pelo menos uma vez. E assim, o uso de memória vai pelo ralo =D

Você já conhece uma solução: Escreva uma `generator function` e a chame. Porém existe uma opção mais fácil: escreva uma `generator expression`. Este é o nome oficial para isso, mas deveria ser chamado realmente de `generator comprehension`. sintaticamente, parece com uma `list comprehension` - tirando o uso dos parentesis ao invés de colchetes.

In [91]:
generated_squares = (n*n for n in range(NUM_SQUARES))
type(generated_squares)

generator

Esta `generator expression` cria um `generator object`, exatamente como a `list comprehension` cria uma lista. Qualquer `list comprehension` que você escreja, pode ser utilizada para criar um `generator object`, somente trocando entre parentesis e colchetes.

E você está criando o objeto diretamente, sem ter que definir uma `generator function` e chama-la. Em outras palavras, uma `generator expression` é um atalho conveniente quando você precisa criar um `generator object` rapidamente:

In [92]:
# Isto...

many_squares = (n*n for n in range(NUM_SQUARES))

# ... é EXATAMENTE igual a isso:

def gen_many_squares(limit):
    
    for n in range(limit):
        
        yield n * n
        
many_squares = gen_many_squares(NUM_SQUARES)

No que diz respeito ao Python, não há diferença.

Tudo que você sabe sobre `list comprehension` aplica-se as `generator expressions`: multiplas clausular `for`, multiplas clausulas `if`, etc. Você só precisa escrever entre parêntesis.

Na verdade, as vezes você pode até mesmo omitir os parentesis. Quando passar a `generator expression` como argumento de uma função, você pode as vezes se ver escrevendo (( "EXPRESSION" )). Nesta situação, Python permite que você omita o par de parentesis interno. Imagine por exemplo, que você está ordenando uma lista e-mails de consumidores, olhando apenas aqueles consumidores os quais o status é ativo:

In [99]:
# User é uma classe com campos email e is_active.
# users é uma lista de objetos do tipo User

class User():
    
    def __init__(self, email, is_active):
        
        self.email = email
        self.is_active = is_active
        
users_emails = [
    'rodrimedeiros@gmail.com', 
    'fred@a.com',
    'sandy@f.net',
    'tim@d.com'
]

users_status = [
    False,
    True,
    True,
    True
]

users = [
    User(email, status) 
    for email, status in zip(users_emails, users_status)
]

# Ordenando por e-mail os elementos gerador por uma
# GENERATOR EXPRESSION
sorted((user.email for user in users if user.is_active))

['fred@a.com', 'sandy@f.net', 'tim@d.com']

In [100]:
# Fazendo a mesma coisa porém omitindo 
# os parentesis internos

sorted(user.email for user in users if user.is_active)

['fred@a.com', 'sandy@f.net', 'tim@d.com']

Observe como legível e natural é (ou vai ser, uma vez que você tiver praticado um pouco). Uma coisa para se atentar: Você somente pode trabalhar com `generator expressions` de uma linha para passar como argumento para funções e métodos sem parentesis. De outra forma, você tem em mãos um erro de sintaxe:

In [101]:
# Exemplo:

sorted(user.email for user in user
      if user.is_active, reverse=True)

SyntaxError: Generator expression must be parenthesized (<ipython-input-101-f2faf818a478>, line 3)

Python não pode interpretar de maneira não ambigua o que você quis dizer aqui, desta forma é necessário utilizar parentesis.

In [102]:
# exemplo que funciona

sorted((user.email for user in users
       if user.is_active), reverse=True)

['tim@d.com', 'sandy@f.net', 'fred@a.com']

E claro, as vezes é mais legível alocar a `generator expression` em uma variável.

In [103]:
active_email = (
    user.email 
    for user in users
    if user.is_active
)

sorted(active_email, reverse=True)

['tim@d.com', 'sandy@f.net', 'fred@a.com']

`Generator expressions` sem parentesis, sugere uma forma forma unificada de pensar sobre `comprehensions`, que conecta `generator expressions` e `list comprehensions`. Aqui está uma `generator expression` para uma sequencia de quadrados:
``` python

(n ** 2 for n in range(10))

```
E aqui é a mesma expressão passada como parâmetro para a built-in list() function:

``` python

list(n ** 2 for n in range(10))

```
E aqui temos uma `list comprehension`:

``` python

[n ** 2 for n in range(10)]

```

Quando você entender `generator expressions`, vai ser fácil enxergar as `list comprehensions` como estruturas de dados derivadas. E o mesmo se aplica para dicionnários e `sets`. Com esta ideia em mente, você começa a ver novas oportunidades de usar tudo isso em seu código, melhorando legibilidade, manutenibilidade e performance.

### `Generator Expression` ou `List Comprehension`?

E as `generator expressions` são tão incríveis, porque usariamos `list comprehensions`? De modo geral, quando decidir qual usar, seu código vai ser mais escalável e responsivo se você utilizar uma `generator expression`. A menos é claro, quando você precisar efetivamente de uma lista. Existem algumas considerações.

Primeiro, se é improvável que a lista seja muito grande - e por grande, quero dizer pelo menos milhares de elementos - você provavelmente não vai se beneficiar muito utilizando uma `generator expression`. simplesmente a dimensão não é grande o bastante para que a escalabilidade importe. As `generator expressions` são imutáveis. Se você quiser acesso randômico, ou passar pela sequencia duas vezes ou se você quiser adicionar e remover elementos, as `generator expressions` não vão funcionar.

__COMENTÁRIO PESSOAL__

Neste ponto falando sobre limitações dos `generator objects`, podemos entender também como limitações dos iteradores.

Isto é especialmente importante escrevendo métodos e funções que retornem uma sequencia. Devemos retornar uma `generator expression` ou uma lista? Em teoria, não há razão para retornar uma lista ao invés de uma `generator object`; Uma lista pode ser trivialmente criada ao passar o `generator object` pela build-in function `list()`. Na prática, a interface pode ser tal que quem chamou a função, realmente queira uma lista. Além disso, se está construindo um valor de retorno como uma lista dentro da função, é bobagem retornar uma `generator expression`, só retorne a lista mesmo.

E se sua intenção é criar uma biblioteca utilizável por pessoas que possam não ser pythonistas avançados, isto pode ser um argumento para retornar listas. Quase todos os programadores são familiares com estruturas de lista. Mas poucos são familiares como `generators` funcionam em python, e podem - e é razoável - ficar um pouco confusos quando confrotados com um `generator object`.

### Dicionários, `Sets` e `Tuples`

Assim como uma `list comprehension` cria uma lista, um `dictionary comprehension` cria um dicionário. Você viu um exemplo no começo do capítulo; aqui vai outro. Suponhaque você tenha uma classe `Student`:

In [109]:
class Student:
    
    def __init__(self, name, gpa, major):
        
        self.name = name
        self.gpa = gpa
        self.major = major

Dado uma lista de objetos da classe `Student`, podemos escrever um  `dictionary comprehension` que mapeia o nome dos estudantes com suas notas:

In [116]:
students_list = [
    ('Jim Smith', 3.6, 'Computer Science'),
    ('Ryan Spencer', 3.1, 'Economics'),
    ('Penny Gilmore', 3.9, 'Computer Science'),
    ('Alisha jones', 2.5, 'Economics'),
    ('Todd Reynolds', 3.4, 'Basket Weaving')
]

students = [
    Student(name, gpa, major)
    for name, gpa, major in students_list
]

In [117]:
{student.name: student.gpa for student in students}

{'Jim Smith': 3.6,
 'Ryan Spencer': 3.1,
 'Penny Gilmore': 3.9,
 'Alisha jones': 2.5,
 'Todd Reynolds': 3.4}

A sintaxe difere da `list comprehension` de duas formas. Ao invés de colchetes, utilizamos chaves - o que faz sentido, já que são as ferramentas utilizadas para criar dicionários. A outra diferença é o campo de expressão, cujo formato é "Chave: Valor", já que o dicionário é composto por elementos neste formato. Então, a estrutura fica da seguinte forma:

{ KEY: VALUE for VARIABLE in SEUQUENCE}

Essas são as únicas diferenças. Todo o resto que foi aprendido sobre `list comprehensions` se aplica aqui também, incluindo as clausulas `if`. 

In [118]:
def invert_name(name):
    
    first, last = name.split(" ", 1)
    
    return ', '.join([last, first])

In [119]:
# Get last name, first name of high-GPA students

{
    invert_name(student.name): student.gpa
    for student in students
    if student.gpa > 3.5    
}

{'Smith, Jim': 3.6, 'Gilmore, Penny': 3.9}

Você pode criar `sets` também. `Set comprehensions` são exatamente como `list comprehensions`, mas ao invés de utilizar colchetes, utilizamos chaves.

In [120]:
# Criando uma lista de cursos...

major_list = [
    student.major for student in students
]

print(major_list)

# Fazendo a mesma operação porém criando um set...

major_set = {
    student.major for student in students
}

print(major_set)

# Também é possível utilizar o conceito de `generator expression`
# em conjunto com a built-in function set()

major_set_builtin = set(student.major for student in students)

print(major_set_builtin)

['Computer Science', 'Economics', 'Computer Science', 'Economics', 'Basket Weaving']
{'Basket Weaving', 'Computer Science', 'Economics'}
{'Basket Weaving', 'Computer Science', 'Economics'}


Como Python faz distinção entre `set` e `dict comprehensions`? Simples! Os dicionários utilizam a expressão "chave: valor", enquanto que os `sets` utilizando as expressões padrão com valores simples.

E sobre as `tuple comprehensions`? Engraçado: Falando diretamente, Python não as suporta - __COMENTÁRIO PESSOAL__ já que os parentesis são utilizados para gerar `generator expressions`- Entretanto, você pode criar uma tuple através da built-in function `tuple()`:

In [122]:
tuple(
    student.gpa 
    for student in students
    if student.major == "Computer Science"
)

(3.6, 3.9)

Esta sentença cria uma `tuple`, mas nã é uma `tuple comprehension`. Você está chamando a função built-in `tuple()` que é um construtor e passando um argumento. E qual é o argumento? Uma `generator expression`!

__COMENTÁRIO PESSOAL__

Isso é lindo demais!

In [125]:
cs_students = (student.gpa for student in students 
              if student.major == 'Computer Science')

print(type(cs_students))

tuple(cs_students)

<class 'generator'>


(3.6, 3.9)

In [126]:
# É a mesmo coisa de:

tuple(
(
    student.gpa for student in students
    if student.major == 'Computer Science'
))

# Mas lembrando do que vimos anteriormente, no caso de passar
# uma generator expression como parametro podemos omitir os pa-
# rentesis internos.

(3.6, 3.9)

O construtor de `tuples`, recebe um iterador como argumento. O `cs_student` é um `generator object` (criado pela `generator expression`), e um `generator object` nada mais é do que um iterador. Desta forma conseguimos ver como Python tem `tuple comprehensions`, utilizando `tuple(generator expression)`. Na verdade, esta construção nos dá alternativas para criação de dicionários e `sets` através de `comprehensions`.

In [127]:
# Alternativa:

dict(
    (student.name, student.gpa)
    for student in students
)

{'Jim Smith': 3.6,
 'Ryan Spencer': 3.1,
 'Penny Gilmore': 3.9,
 'Alisha jones': 2.5,
 'Todd Reynolds': 3.4}

In [128]:
# Alternativa:

set(student.major for student in students)

{'Basket Weaving', 'Computer Science', 'Economics'}

Lembrando que, quando você passa uma `generator expression` como parametro de uma funlão, você pode omitir os parentesis internos. Por isso, podemos por exemplo escrever:

```python

tuple(f(x) for x in numbers)

```

ao invés de 

```python

tuple((f(x) for x in numbers))

```

Um último ponto. `Generator expressions` são são análogos escaláveis das `list comprehensions`; Existe algo equivalente para `dicts` e `sets`? Não! Não tem. Se você precisar gerar preguiçosamente pares chave-valor de elementos únicos, sua melhor aposta é criar uma `generator function`.

### Limites das `Comprehensions`

As `comprehensions` tem alguns pontos que as vezes as pessoas tropeçam. Considere o seguinte código.

In [148]:
# O objetivo é ler as linhas de um arquivo, removendo espaços em
# branco e desconsiderando linhas vazias ou que contenham somente
# espaços.

trimmed_lines = list()

for line in open(r'src/wombat-story.txt'):
    
    line = line.strip()
    
    if line != '':
        
        trimmed_lines.append(line)
        
print(f"Got {len(trimmed_lines)} ")

Got 9 


__COMENTÁRIO PESSOAL:__

o método `.strip()` remove espaços em branco no inicio e final da string. Tem como default um parâmetro None com isso remove espaços, se ao invés de None for passado uma cadeia de caracteres, estes caracteres serão removidos do inicio e fim da string.

Sendo bem direto, o que estamos construindo é uma lista chamada `trimmed_lines`. A lista resultante, tem os espaços em branco iniciais e finais removidos de cada linha, e ignora linhas varias ou compostas somente por espaços. Não é difícil imaginar a necessidade de fazer algo do tipo em um programa real. 

Agora, como poderiamos fazer isso utilizando uma `list comprehension`?

In [149]:
trimmed_lines = [
    line.strip()
    for line in open(r'src/wombat-story.txt')
    if line.strip() != ''
]

print(f"Got {len(trimmed_lines)} ")

Got 9 


Isto funciona. Mas note que a instrução `line.strip()` aparece duas vezes. Isso faz com que CPU seja despesdiçado comparado com a versão com o loop `for` que executa este comando apenas uma vez. Remover espaços de uma string não é uma operação muito custosa, computacionalmente falando. Mas cedo ou tarde, você vai querer fazer algo onde essa duplicação importa:

```python

values = [
    expensive_function(n)
    for n in range(BIG_NUMBER)
    if expensive_function(n) > 0
]

```

Então, como criar neste caso, uma `list comprehension`, chamando a função `expensive_function` apenas uma vez? A notícia ruim é que não há uma maneira direta de fazer isso. Existem alguma soluções bem inteligentes para resolver isso, como por exemplo `memoizing` (o que pode facilmente consumir mais memória do que o preciso), definir uma `generator expression` (provavelmente a melhor escolha) ou fazer uma lista intermediária (se ela for pequena). Se a sequencia que você precisa, segue o padrão acima, você pode considerar construi-la da forma antiga com `for loops`, ou uilizar uma `generator function`. Ou se mesmo assim você ainda quiser utilizar uma `comprehension`, use uma `generator expression` no nível intermediário. O resultado é bem legível e tranquilo de entender (pelo menos pra você, que acompanhou o livro até agora).


```python

intermediate_values = (
    expensive_function(n)
    for n in range(10000)
)


values = [
    intermediate_values
    for intermediate_value in intermediate_values
    if intermediate_value > 0
]

```

__COMENTÁRIO PESSOAL__

Neste caso, através de uma `generator expression` intermediária, pudemos calcular a expressão computacionalmente custosa apenas uma vez.

- Definição de __memoizing__: Pelo que pesquisei, memoizing é quando você faz um cálculo custoso e guarda as respostas em cache, quando o cálculo for necessário novamente, ao invés de fazer o cálculo novamente, o programa pega a resposta previamente calculada. 

Outra limitação, é que as `list comprehensions` são construidas um elemento de cada vez. A melhor forma de enxergar isso é imaginar uma lista composta por pares chave-valor em linha. Em outras palavras, os elementos chave ocorrem com seus valores logo em seguida. Imagine uma função para converter esta lista em um dicionário.

In [150]:
# preço por kg de frutas e vegetais em dolar

prices_flat_list = [
    'orange', 0.70,
    'banana', 0.86,
    'cantaloupe', 0.63,
    'bok choy', 1.56,
    'coconuts', 1.06
]

def list2dict(flattened):
    
    assert len(flattened) % 2 == 0, "Input must be list of key-value pairs"
    
    unflattened = dict()
    
    for i in range(0, len(flattened), 2):
        
        key, value = flattened[i], flattened[i + 1]
        
        unflattened[key] = value
    
    return unflattened       

list2dict(prices_flat_list)

{'orange': 0.7,
 'banana': 0.86,
 'cantaloupe': 0.63,
 'bok choy': 1.56,
 'coconuts': 1.06}

Olhe para o loop na função `list2dict`. Ele percorre os índices pares, da lista passada (`flattened`) ao invés de acessar diretamente seu elementos. Isto permite que possamos referenciar dois elementos por vez (por cada iteração do loop). Mas isto acaba sendo algo que não pode ser expresso na semantica das `list comprehension`. Geralmente, uma `comprehension` opera olhando cada elemento de alguma sequencia fonte por vez; Ela não pode inspecionar elementos vizinhos.

__COMENTÁRIO PESSOAL__

Vou tentar fazer a implementação desde exemplo através de um `dict comprehension`

In [152]:
{
    prices_flat_list[i]: prices_flat_list[i + 1]
    for i in range(0, len(prices_flat_list), 2)
}

{'orange': 0.7,
 'banana': 0.86,
 'cantaloupe': 0.63,
 'bok choy': 1.56,
 'coconuts': 1.06}

No caso aqui, eu consegui fazer um `dict comprehension` acessando através do índice, não acessando o elemento diretamente, talvez a limitição seja com relação a acessar o elemento diretamente.

Outro exemplo: Uma função que agrupa os elementos de uma sequencia de acordo com algum critério - por exemplo, a primeira letra de uma string.

In [159]:
from collections import defaultdict

def group_by_first_letter(items):
    
    grouped = defaultdict(list)
    
    for item in items:
        
        key = item[0].lower()
        grouped[key].append(item)
        
    return dict(grouped)

__COMENTÁRIO PESSOAL__

bem legal a utilização do `defaultdict` pelo que entendi é um dict que carrega valores default para qualquer chave que venha a ser inserida, no caso inicializo a chave com uma lista vazia. Vou fazer um teste rápido inicializando com zero pois daí posso fazer um esquema de montar um contador.

In [172]:
names = 'rodrigo rodrigo rodrigo renata renata \
         renata renata isabella \
         isabella isabella gabi gabi gabi gabi gabi'.split()

In [173]:
def return_zero():
    
    return 0

name_counter = defaultdict(return_zero)

for name in names:
    
    name_counter[name] += 1
    

name_counter = dict(name_counter)
name_counter

{'rodrigo': 3, 'renata': 4, 'isabella': 3, 'gabi': 5}

Muito legal, funcionou o contador com o `defaultdict` lembrando que ele precisa receber um `callable`, um objeto que possa ser chamado (funções, objetos...). Com isso ao invés de passar o valor zero diretamente, criei uma função que simplesmente retorna zero. Com isso todas as chaves agora passaram a ser inicializadas com zero. Posso também fazer uma função anonima será que funciona? Vamos ver:

In [174]:
name_counter = defaultdict(lambda :0)

for name in names:
    
    name_counter[name] += 1
    

name_counter = dict(name_counter)
name_counter

{'rodrigo': 3, 'renata': 4, 'isabella': 3, 'gabi': 5}

In [160]:
names = 'Joe Jim Todd Tiffany Zelma Gerry Gina'.split()

grouped_names = group_by_first_letter(names)

In [162]:
grouped_names

{'j': ['Joe', 'Jim'],
 't': ['Todd', 'Tiffany'],
 'z': ['Zelma'],
 'g': ['Gerry', 'Gina']}

Novamente, a semantica das `Python comprehensions` não foram feitas para suportar este tipo de algoritmo. Em termos de programação funcional, `comprehensions` pode usar operações de `map` e `filter`, mas não podem utilizar `reduce` ou `fold`. Felizmente, isto cobre a maioria dos casos. Apontei essas limitações para evitar que você gaste tempo tentando descobri-las.  

In [12]:
def double_short_words(words):
    
    return [
        word + word
        for word in words
        if len(word) < 5
    ]

In [16]:
words = 'ola ole ovo rodrigo bernardo medeiros'.split()

double_small_words = double_short_words(words)
print(double_small_words)

['olaola', 'oleole', 'ovoovo']
